# **Exercises**

Do you work for this exercise in either a jupyter notebook named evaluate within your regression-exercises repository. By the end of this exercise you will also create a python script named evaluate, so the overall deliverables for this exercise are the python script and jupyter notebook both with the name evaluate within your regression-exercises repo.



In [9]:
import sklearn.preprocessing

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr, spearmanr

from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, QuantileTransformer

#import my modules
import acquire as a
import prepare as p
import wrangle as w

from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression 
from math import sqrt


**Q1) Load your zillow dataset.**

In [2]:
df = w.wrangle_zillow()
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
4,4,2.0,3633.0,296425.0,2005,6941.39,6037
6,3,4.0,1620.0,847770.0,2011,10244.94,6037
7,3,2.0,2077.0,646760.0,1926,7924.68,6037
18,3,1.0,1244.0,169471.0,1950,2532.88,6037
19,3,2.0,1300.0,233266.0,1950,3110.99,6037


In [3]:
train_validate, test = train_test_split(df, test_size=.2, random_state=123)
train, validate = train_test_split(train_validate, test_size=.3, random_state=123)
print(train.shape, validate.shape, test.shape)

(1161886, 7) (497952, 7) (414960, 7)


**Q2) Fit a linear regression model (ordinary least squares) and compute yhat, predictions of taxvaluedollarcnt using only calfulatedfinishedsqft.**

In [4]:
df1 = train[['taxvaluedollarcnt', 'calculatedfinishedsquarefeet']]

In [5]:
df1.head()

,taxvaluedollarcnt,calculatedfinishedsquarefeet
775411,330163.0,1692.0
1160935,631368.0,2119.0
1558975,571105.0,1363.0
2049378,321859.0,1348.0
1250609,307370.0,1635.0


In [6]:
df1['yhat_baseline'] = df['calculatedfinishedsquarefeet'].mean()
df1.head(3)

,taxvaluedollarcnt,calculatedfinishedsquarefeet,yhat_baseline
775411,330163.0,1692.0,1895.158555
1160935,631368.0,2119.0,1895.158555
1558975,571105.0,1363.0,1895.158555


In [8]:
# generate parameters, i.e. create model
ols_model = LinearRegression().fit(df1[['taxvaluedollarcnt']], df1.calculatedfinishedsquarefeet)

# compute predictions and add to original dataframe
df1['yhat'] = ols_model.predict(df1[['taxvaluedollarcnt']])

df1.head(3)


,taxvaluedollarcnt,calculatedfinishedsquarefeet,yhat_baseline,yhat
775411,330163.0,1692.0,1895.158555,1777.448478
1160935,631368.0,2119.0,1895.158555,2034.990406
1558975,571105.0,1363.0,1895.158555,1983.463210


**Q3) Plot the residuals for the linear regression model that you made.**

In [11]:
# compute residuals
df1['residual'] = df1['yhat'] - df1['calculatedfinishedsquarefeet']
df1['residual_baseline'] = df1['yhat_baseline'] - df1['calculatedfinishedsquarefeet']


In [12]:
df1.head()

,taxvaluedollarcnt,calculatedfinishedsquarefeet,yhat_baseline,yhat,residual,residual_baseline
775411,330163.0,1692.0,1895.158555,1777.448478,85.448478,203.158555
1160935,631368.0,2119.0,1895.158555,2034.990406,-84.009594,-223.841445
1558975,571105.0,1363.0,1895.158555,1983.463210,620.463210,532.158555
2049378,321859.0,1348.0,1895.158555,1770.348237,422.348237,547.158555
1250609,307370.0,1635.0,1895.158555,1757.959582,122.959582,260.158555


viz.evaluation_example5(df, x, df.residual_baseline)
viz.evaluation_example5(df, x, df.residual)

In [13]:
# square each residual value
df1['residual^2'] = df1.residual ** 2

df1['residual_baseline^2'] = df1.residual_baseline ** 2

df1.head(3)


,taxvaluedollarcnt,calculatedfinishedsquarefeet,yhat_baseline,yhat,residual,residual_baseline,residual^2,residual_baseline^2
775411,330163.0,1692.0,1895.158555,1777.448478,85.448478,203.158555,7301.442450,41273.398353
1160935,631368.0,2119.0,1895.158555,2034.990406,-84.009594,-223.841445,7057.611832,50104.992628
1558975,571105.0,1363.0,1895.158555,1983.463210,620.463210,532.158555,384974.594793,283192.727353


In [14]:
# SSE
SSE = sum(df1['residual^2'])
SSE_baseline = sum(df1['residual_baseline^2'])

print("SSE = ", SSE)
print("SSE - baseline = ", SSE_baseline)


SSE =  1852870397904.066
SSE - baseline =  2252368212272.5786


In [15]:
MSE = SSE/len(df1)
MSE_baseline = SSE_baseline/len(df1)

print("MSE = ", MSE)
print("MSE baseline = ", MSE_baseline)


MSE =  1594709.289813343
MSE baseline =  1938544.9280502377


In [16]:
from math import sqrt
RMSE = sqrt(MSE)
RMSE_baseline = sqrt(MSE_baseline)

print("RMSE = ", RMSE)
print("RMSE baseline = ", RMSE_baseline)


RMSE =  1262.81799552166
RMSE baseline =  1392.3163893491442


My modle is working better but is still of by a large sum.  The model does better than having no model at all.

In [18]:
from sklearn.metrics import mean_squared_error

SSE2 = mean_squared_error(df1.calculatedfinishedsquarefeet, df1.yhat)*len(df1)
SSE2_baseline = mean_squared_error(df1.calculatedfinishedsquarefeet, df1.yhat_baseline)*len(df1)

print("SSE manual == SSE sklearn: ", SSE == SSE2) 
print("SSE manual - baseline == SSE sklearn - baseline: ", SSE_baseline == SSE2_baseline) 


SSE manual == SSE sklearn:  False
SSE manual - baseline == SSE sklearn - baseline:  False


In [19]:
MSE2 = mean_squared_error(df1.calculatedfinishedsquarefeet, df1.yhat)

MSE2_baseline = mean_squared_error(df1.calculatedfinishedsquarefeet, df1.yhat_baseline)


print("MSE manual == MSE sklearn: ", MSE == MSE2) 
print("MSE manual - baseline == MSE sklearn - baseline: ", MSE_baseline == MSE2_baseline) 


MSE manual == MSE sklearn:  False
MSE manual - baseline == MSE sklearn - baseline:  False


In [20]:
RMSE2 = sqrt(mean_squared_error(df1.calculatedfinishedsquarefeet, df1.yhat))
RMSE2_baseline = sqrt(mean_squared_error(df1.calculatedfinishedsquarefeet, df1.yhat_baseline))

print("RMSE manual == RMSE skearn: ", RMSE == RMSE2) 
print("RMSE manual - baseline == RMSE skearn - baseline: ", RMSE_baseline == RMSE2_baseline) 


RMSE manual == RMSE skearn:  False
RMSE manual - baseline == RMSE skearn - baseline:  False


In [21]:
df_eval = pd.DataFrame(np.array(['SSE','MSE','RMSE']), columns=['metric'])
df_baseline_eval = pd.DataFrame(np.array(['SSE_baseline','MSE_baseline','RMSE_baseline']), columns=['metric'])

df_eval['model_error'] = np.array([SSE, MSE, RMSE])
df_baseline_eval['model_error'] = np.array([SSE_baseline, MSE_baseline, RMSE_baseline])

print(df_eval)
print(df_baseline_eval)


  metric   model_error
0    SSE  1.852870e+12
1    MSE  1.594709e+06
2   RMSE  1.262818e+03
          metric   model_error
0   SSE_baseline  2.252368e+12
1   MSE_baseline  1.938545e+06
2  RMSE_baseline  1.392316e+03


Draw Conclusions

Now, we will use our results to select the best model.

We will compare each baseline metric with the respective metrics of the linear regression model to see if our model performs better.

In [22]:
df_eval['error_delta'] = df_eval.model_error - df_baseline_eval.model_error
df_eval


,metric,model_error,error_delta
0,SSE,1.852870e+12,-3.994978e+11
1,MSE,1.594709e+06,-3.438356e+05
2,RMSE,1.262818e+03,-1.294984e+02


**Evaluate Part 2: Model Significance**

In [23]:
# compute explained sum of squares
ESS = sum((df1.yhat - df.calculatedfinishedsquarefeet.mean())**2)


In [25]:
SSE = df_eval[df_eval.metric=='SSE']['model_error']
TSS = ESS + SSE
print("ESS = ", ESS)
print("SSE = ", SSE[0])
print("TSS = ", TSS[0])


ESS =  399497814368.00433
SSE =  1852870397904.066
TSS =  2252368212272.0703


In [27]:
# add the ESS, SSE, TSS from the model to a dataframe
ss = pd.DataFrame(np.array(['SSE','ESS','TSS']), columns=['metric'])
ss['model_values'] = np.array([SSE[0], ESS, TSS[0]])

# compute and add the baseline values to the dataframe
ESS_baseline = sum((df1.yhat_baseline - df1.calculatedfinishedsquarefeet.mean())**2)
SSE_baseline = df_baseline_eval[df_baseline_eval.metric=='SSE_baseline']['model_error'][0]
TSS_baseline = ESS_baseline + SSE_baseline


In [28]:
ss['model_values'] = np.array([SSE_baseline, ESS_baseline, TSS_baseline])

ss


,metric,model_values
0,SSE,2.252368e+12
1,ESS,1.035431e+05
2,TSS,2.252368e+12


In [29]:
R2 = ESS/TSS
print('R-squared = ',round(R2,3))
print("Percent of variance in y explained by x = ", round(R2*100,1), "%")


R-squared =  0    0.177
Name: model_error, dtype: float64
Percent of variance in y explained by x =  0    17.7
Name: model_error, dtype: float64 %


In [31]:
# sklearn.metrics.explained_variance_score

evs = explained_variance_score(df1.calculatedfinishedsquarefeet, df1.yhat)
print('Explained Variance = ', round(evs,3))


Explained Variance =  0.177


Calculate the sum of squared errors, explained sum of squares, total sum of squares, mean squared error, and root mean squared error for your model.

Calculate the sum of squared errors, mean squared error, and root mean squared error for the baseline model (i.e. a model that always predicts the average taxvaluedollarcnt amount).

Write python code that compares the sum of squared errors for your model against the sum of squared errors for the baseline model and outputs whether or not your model performs better than the baseline model.

What is the amount of variance explained in your model?

Is your model better than the baseline model?

Create a file named evaluate.py that contains the following functions.

plot_residuals(y, yhat): creates a residual plot
regression_errors(y, yhat): returns the following values:
sum of squared errors (SSE)
explained sum of squares (ESS)
total sum of squares (TSS)
mean squared error (MSE)
root mean squared error (RMSE)
baseline_mean_errors(y): computes the SSE, MSE, and RMSE for the baseline model
better_than_baseline(y, yhat): returns true if your model performs better than the baseline, otherwise false
Bonus Exercises

